In [91]:
# TODO: constraints; cost function? why nega score?; goal is to achieve comparable performance. 

In [1]:
# https://www.kaggle.com/pavelvod/pytorch-starter-solution
!pip install torch

    100% |████████████████████████████████| 71.1MB 657kB/s ta 0:00:011


In [120]:
import utils
import importlib

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
# giftCostsNp (5000, 100): [i,j] means the gift cost when i-th family choses j-th day.
row, col = 5000, 100
giftCostsNp = np.zeros((row, col))
familySizesNp = np.zeros((row, 1))

familyDataNp = utils.readDataDf().values

for i in range(row):
    familySizesNp[i] = familyDataNp[i, -1]
    for j in range(col):
        giftCostsNp[i,j] = utils.getCurGift(assignDay=j, 
                                          familyPreferencesList=familyDataNp[i, 1:-1].tolist(), 
                                          familySize=familySizesNp[i])

In [216]:
class Model(nn.Module):
    def __init__(self, giftCostsNp, familySizesNp):
        super().__init__()
        self.giftCostsTc = torch.from_numpy(giftCostsNp).type(torch.float32)
        self.familySizesTc = torch.from_numpy(familySizesNp).type(torch.float32)
        self.assignWeightsTc = torch.nn.Parameter(data=torch.Tensor(5000, 100), requires_grad=True)
        self.assignWeightsTc.data.uniform_(0,5) # weight initialization?
        
    def forward(self):
        assignProbTc = F.softmax(self.assignWeightsTc, dim=1)
#         print(self.giftCostsTc.shape, assignProbTc.shape)
        giftCostTotalTc = (self.giftCostsTc * assignProbTc).sum()
        
        NdTc = (assignProbTc.transpose(0,1) @ self.familySizesTc).sum(axis=1)
#         print(F.softmax(NdTc).max(), F.softmax(NdTc).argmax())
        NdTcCat = torch.zeros(101, dtype=torch.float32)
        NdTcCat[:-1] = NdTc
        NdTcCat[-1] = NdTc[-1]
        
#         print("exponent: ", 0.5+abs(NdTcCat[:-1]-NdTcCat[1:])/50.)
        cleanCostTotalTc = ((NdTc-125.)/400. * NdTc**(0.5+abs(NdTcCat[:-1]-NdTcCat[1:])/50.)).sum()
        
        penaltyCostTotalTc = (torch.relu(125.-NdTc)**2 + torch.relu(NdTc-300.)**2).sum() * 1000000
        return giftCostTotalTc, cleanCostTotalTc, penaltyCostTotalTc

In [217]:
# def cost_function():
#     return None
importlib.reload(utils)

<module 'utils' from '/Users/songxu/PycharmProjects/Data_Science/Kaggle/santa-workshop-tour-2019/utils.py'>

In [218]:
model = Model(giftCostsNp, familySizesNp)
best_score = 10e10
best_pos = None

In [219]:
optimizer = torch.optim.Adam(model.parameters())
for epoch in range(150000):
    optimizer.zero_grad()
    x, y, z = model()
#     print(x.shape, y.shape)
    loss = x + y + z
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        # (5000, 100): for each family (keep dim 0), find the most-weighted day
        assignments = model.assignWeightsTc.argmax(dim=1).numpy()+1
#         print(model.assignWeightsTc[0,:])
#         print(model.assignWeightsTc[0,:].max(), model.assignWeightsTc[0,:].argmax())
        
#         print(assignments.max(), assignments.min())
        curScore = utils.getTotalScore(assignments)
#         print(x, y, z)
        print(curScore, z, loss)
#         break
#         pos = model.assignWeightsTc.argmax(1).numpy()
#         a, b, v = cost_function(pos+1)
#         score = a + b
#         if score < best_score:
#             best_score = score
#             best_pos = pos
#         x = np.round(x.item(), 3)
#         y = np.round(y.item(), 3)
#         print(f"{epoch}\t{x}\t{y}\t{score}\t{a}\t{b}\t{v}")

10620037 3574531.8473585383
14194568.847358538 tensor(0., grad_fn=<MulBackward0>) tensor(10649078., grad_fn=<AddBackward0>)
10612308 1.1879714001884732e+20
1.1879714001885793e+20 tensor(0., grad_fn=<MulBackward0>) tensor(10458506., grad_fn=<AddBackward0>)
10633174 4.7105822958330924e+27
4.7105822958330924e+27 tensor(0., grad_fn=<MulBackward0>) tensor(10221365., grad_fn=<AddBackward0>)
10575880 6.816053820153423e+22
6.816053820153424e+22 tensor(0., grad_fn=<MulBackward0>) tensor(9929650., grad_fn=<AddBackward0>)
10546788 8.224740658125752e+19
8.224740658126807e+19 tensor(0., grad_fn=<MulBackward0>) tensor(9574804., grad_fn=<AddBackward0>)
10527965 6.833060205689674e+16
6.83306020674247e+16 tensor(0., grad_fn=<MulBackward0>) tensor(9215375., grad_fn=<AddBackward0>)
10542625 48684799230405.05
48684809773030.05 tensor(0., grad_fn=<MulBackward0>) tensor(8809120., grad_fn=<AddBackward0>)
10536396 15931736145324.297
15931746681720.297 tensor(0., grad_fn=<MulBackward0>) tensor(8372946.5000, gr

KeyboardInterrupt: 